In [ ]:
import keras
from tensorflow.keras.applications import ResNet50V2
import zipfile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/Bugs.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

dataGenerator = keras.preprocessing.image.ImageDataGenerator(validation_split = 0.3, samplewise_center = True)

test = dataGenerator.flow_from_directory("/tmp/Bugs", class_mode='categorical', target_size = (64,64), batch_size=32, subset="validation", shuffle=True)
train = dataGenerator.flow_from_directory("/tmp/Bugs", class_mode='categorical', target_size = (64,64), batch_size=32, subset="training", shuffle=True)

Found 227 images belonging to 3 classes.
Found 537 images belonging to 3 classes.


In [ ]:
def bugNet():
  
    model = keras.models.Sequential()
    model.add(keras.layers.experimental.preprocessing.RandomRotation(1,input_shape = (64,64,3)))
    model.add(keras.layers.experimental.preprocessing.RandomFlip("horizontal"))
    model.add(keras.layers.experimental.preprocessing.RandomFlip("vertical"))
    model.add(keras.layers.experimental.preprocessing.RandomZoom(0.2))
    model.add(keras.layers.experimental.preprocessing.RandomTranslation(height_factor = 0.1, width_factor = 0.1))
    model.add(ResNet50V2(include_top = False, weights = "imagenet", input_tensor = keras.Input(shape = (64, 64, 3))))
    model.add(keras.layers.GlobalAveragePooling2D())
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(256, activation = "relu"))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(3, activation = "softmax"))
    
    #iterationsToChangeAt = [60,120]
    #LRs = [.001, .0001, 0.00001]
    #stepRate = keras.optimizers.schedules.PiecewiseConstantDecay(iterationsToChangeAt, LRs)

    # Compile the model
    model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.SGD(learning_rate = 0.001), metrics=["categorical_accuracy"])
    
    return(model)
    
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau

mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_delta=1e-4, mode='min')

model = bugNet()
model.summary()
model.fit(train, epochs=150, validation_data=test, callbacks=[mcp_save, reduce_lr_loss])

#IMPORTANT: Your model must be saved as "Q3.h5"
model.save("./Q2.h5")

94674944/94668760 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_rotation (RandomRotat (None, 64, 64, 3)         0         
_________________________________________________________________
random_flip (RandomFlip)     (None, 64, 64, 3)         0         
_________________________________________________________________
random_flip_1 (RandomFlip)   (None, 64, 64, 3)         0         
_________________________________________________________________
random_zoom (RandomZoom)     (None, 64, 64, 3)         0         
_________________________________________________________________
random_translation (RandomTr (None, 64, 64, 3)         0         
_________________________________________________________________
resnet50v2 (Functional)      (None, 2, 2, 2048)        23564800  
_________________________________________________________

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


17/17 [==============================] - ETA: 0s - loss: 1.8771 - categorical_accuracy: 0.3285

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


17/17 [==============================] - 39s 1s/step - loss: 1.8716 - categorical_accuracy: 0.3288 - val_loss: 7.8149 - val_categorical_accuracy: 0.3084
Epoch 2/150
17/17 [==============================] - 18s 1s/step - loss: 1.5647 - categorical_accuracy: 0.3504 - val_loss: 3.5289 - val_categorical_accuracy: 0.3348
Epoch 3/150
17/17 [==============================] - 18s 1s/step - loss: 1.3126 - categorical_accuracy: 0.3414 - val_loss: 2.0954 - val_categorical_accuracy: 0.3965
Epoch 4/150
17/17 [==============================] - 18s 1s/step - loss: 1.1439 - categorical_accuracy: 0.4435 - val_loss: 1.5529 - val_categorical_accuracy: 0.4229
Epoch 5/150
17/17 [==============================] - 18s 1s/step - loss: 1.0887 - categorical_accuracy: 0.4085 - val_loss: 1.3579 - val_categorical_accuracy: 0.4185
Epoch 6/150
17/17 [==============================] - 18s 1s/step - loss: 1.1163 - categorical_accuracy: 0.4238 - val_loss: 1.2239 - val_categorical_accuracy: 0.4317
Epoch 7/150
17/17 [===